In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import WUBRG
import consts
from FormatMetadata import SETS, FORMATS, SET_CONFIG
from JSONHandler import JSONHandler
from FormatMetadata import FormatMetadata

In [2]:
from datetime import date, time, datetime, timedelta
import pandas as pd

import WUBRG
import consts
from JSONHandler import JSONHandler

class RawDataHandler:    
    def __init__(self, SET, FORMAT):
        self._SET = SET
        self._FORMAT = FORMAT
        self.FORMAT_METADATA = FormatMetadata(SET, FORMAT)
        
        self._SUMMARY_DICT = dict()
        self._ARCHTYPE_DICT = dict()
        self._CARD_DICTS = dict()
        
        self.SUMMARY_FRAME = None
        self.ARCHTYPE_FRAME = None
        self.CARD_FRAME = None
    
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
    
    
    @property
    def FORMAT(self):
        """The queue type."""
        return self._FORMAT
    
    
    def panadafy_card_dict(self, card_dict: dict) -> pd.DataFrame:
        """
        Turns a dictionary into a DataFrame, with some data cleaning applied.
        :param card_dict: The dictionary containing card data for a colour group
        :return: A DataFrame filled with the cleaned card data
        """
        frame = pd.DataFrame.from_dict(card_dict)
        frame = frame.rename(columns=consts.STAT_NAMES)

        # If there's no data, make a blank frame and return it.
        if len(card_dict) == 0:
            return frame

        frame = frame.set_index('Name')
        
        for col in ["GP WR", "OH WR", "GD WR", "GIH WR", "GND WR", "IWD"]:
            frame[col] = frame[col] * 100

        frame = frame.drop(['sideboard_game_count', 'sideboard_win_rate', 'url', 'url_back'], axis=1)
        frame['Rarity'] = frame['Rarity'].map(consts.RARITY_ALIASES)
        frame = frame.round(3)
        return frame
    
    def panadafy_meta_dict(self, meta_dict: dict) -> pd.DataFrame:
        """
        Turns a dictionary into a DataFrame, with some data cleaning applied.
        :param card_dict: The dictionary containing card data for a colour group
        :return: A DataFrame filled with the cleaned card data
        """
        frame = pd.DataFrame.from_dict(meta_dict)
        frame = frame.rename(columns=consts.META_COLS)

        # If there's no data, make a blank frame and return it.
        if len(meta_dict) == 0:
            return frame, frame.copy()
        
        frame['Name'] = frame['Color Name']
        frame = frame.set_index('Name')
        frame['Win %'] = round((frame['Wins'] / frame['Games']) * 100, 2)
        frame['Splash'] = frame['Color Name'].str.contains("Splash", case=False)
        frame['Colors'] = frame['Color Name'].map(lambda x: x.replace(' + Splash', ''))
        frame = frame[['Colors', 'Splash', 'Wins', 'Games', 'Win %', 'is_summary']]
        
        summary_frame = frame[frame['is_summary'] == True].copy()
        summary_frame = summary_frame.drop(['is_summary'], axis=1)
        summary_frame['Colors'] = summary_frame['Colors'].map(WUBRG.COLOR_COUNT_MAP)

        archetype_frame = frame[frame['is_summary'] == False].copy()
        archetype_frame = archetype_frame.drop(['is_summary'], axis=1)
        archetype_frame['Colors'] = archetype_frame['Colors'].map(lambda x: x[0: (x.find('(') if x.find('(') != -1 else len(x))].strip())
        archetype_frame['Colors'] = archetype_frame['Colors'].map(lambda x: x.replace('Mono-', ''))
        archetype_frame['Colors'] = archetype_frame['Colors'].map(WUBRG.COLOR_ALIASES)
        archetype_frame['Name'] = archetype_frame['Colors']
        archetype_frame = archetype_frame.set_index('Name')

        return summary_frame, archetype_frame
    
    def gen_frames(self):
        self.SUMMARY_FRAME = pd.concat(self._SUMMARY_DICT, names=["Date", "Name"])
        self.ARCHTYPE_FRAME = pd.concat(self._ARCHTYPE_DICT, names=["Date", "Name"])
        temp_dict = dict()
        for date in self._CARD_DICTS:
            temp_dict[date] = pd.concat(self._CARD_DICTS[date], names=["Deck Colors", "Name"])
        self.CARD_FRAME = pd.concat(temp_dict, names=["Date", "Deck Colors", "Name"])
        
    def get_day_data(self, check_date):
        loader = JSONHandler(self.SET, self.FORMAT, check_date)
        str_date = str(check_date)
        print(f'Getting data for {self.SET} {self.FORMAT}, date: {str_date}')
        card_dict, meta_dict = loader.get_day_data()
        
        self._SUMMARY_DICT[str_date], self._ARCHTYPE_DICT[str_date] = self.panadafy_meta_dict(meta_dict)
        self._CARD_DICTS[str_date] = dict()
        for color in card_dict:
            self._CARD_DICTS[str_date][color] = self.panadafy_card_dict(card_dict[color])
            
        return self._SUMMARY_DICT[str_date], self._ARCHTYPE_DICT[str_date], self._CARD_DICTS[str_date]

    def get_set_data(self):
        check_date = self.FORMAT_METADATA.START_DATE

        run = True        
        while(run):
            if self.FORMAT_METADATA.is_active(check_date):
                self.get_day_data(check_date)
            check_date += timedelta(days=1)
            run = check_date < date.today() 
    
        self.gen_frames()
        return self._SUMMARY_DICT, self._ARCHTYPE_DICT, self._CARD_DICTS
    
    def get_summary_data(self):
        loader = JSONHandler(self.SET, self.FORMAT, None)
        print(f'Getting overall data for {self.SET} {self.FORMAT}')
        # TODO: Determine when to update based on a time-stamp.
        # NOTE: This might not be necessary, as there isn't a need to automatically update. It may be better
        # to call this ad-hoc.
        card_dict, meta_dict = loader.get_day_data(overwrite=True)
        
        summary, archetype = self.panadafy_meta_dict(meta_dict)
        cards = dict()
        for color in card_dict:
            cards[color] = self.panadafy_card_dict(card_dict[color])
            
        return summary, archetype, cards

In [3]:
class RawDataWrapper:   
    def __init__(self, SET, FORMAT):
        self._SET = SET
        self._FORMAT = FORMAT
        self._DATA = RawDataHandler(SET, FORMAT)
    
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
    
    
    @property
    def FORMAT(self):
        """The format type."""
        return self._FORMAT
    
    
    @property
    def DATA(self):
        """The data about the set and format."""
        return self._DATA
    

    def get_set_data(self):
        self.DATA.get_set_data()
        
    def get_summary_data(self):
        return self.DATA.get_summary_data()
    
    def summary_frame(self, date=slice(None, None, None), name=slice(None, None, None)):
        return self.DATA.SUMMARY_FRAME.loc(axis=0)[pd.IndexSlice[date, name]]
     
    def archetype_frame(self, date=slice(None, None, None), name=slice(None, None, None)):
        return self.DATA.ARCHTYPE_FRAME.loc(axis=0)[pd.IndexSlice[date, name]]
    
    def card_frame(self, date=slice(None, None, None), colors=slice(None, None, None), name=slice(None, None, None)):
        return self.DATA.CARD_FRAME.loc(axis=0)[pd.IndexSlice[date, colors, name]]

In [4]:
class DataManager:
    def __init__(self):        
        #TODO: Get the sets from a config file.
        self.SETS = ['NEO']
        self.FORMATS = ['PremierDraft', 'TradDraft', 'QuickDraft']
        self.DATA = {s: {f: RawDataWrapper(s, f) for f in self.FORMATS} for s in self.SETS}
        
        #TODO: Use properties to better validate set and format values.
        self._ACTIVE_SET = self.SETS[0]
        self._ACTIVE_FORMAT = self.FORMATS[0]
        self._ACTIVE_DATA = self.DATA[self.ACTIVE_SET][self.ACTIVE_FORMAT]
        
    @property
    def ACTIVE_SET(self):
        """The draft set."""
        return self._ACTIVE_SET

    @ACTIVE_SET.setter
    def ACTIVE_SET(self, value):
        if value in SETS:
            self._ACTIVE_SET = value
            self._ACTIVE_DATA = self.DATA[self.ACTIVE_SET][self.ACTIVE_FORMAT]
        else:
            raise ValueError(f"Inavlid set: '{value}'")
        
    @property
    def ACTIVE_FORMAT(self):
        """The draft set."""
        return self._ACTIVE_FORMAT

    @ACTIVE_FORMAT.setter
    def ACTIVE_FORMAT(self, value):
        if value in FORMATS:
            self._ACTIVE_FORMAT = value
            self._ACTIVE_DATA = self.DATA[self.ACTIVE_SET][self.ACTIVE_FORMAT]
        else:
            raise ValueError(f"Inavlid format: '{value}'")
            
    
    @property
    def ACTIVE_DATA(self):
        """The data for the given set and format."""
        return self._ACTIVE_DATA
    
    
    def summary_frame(self, date=slice(None, None, None), name=slice(None, None, None)):
        return self.ACTIVE_DATA.summary_frame(date, name)
     
    def archetype_frame(self, date=slice(None, None, None), name=slice(None, None, None)):
        return self.ACTIVE_DATA.archetype_frame(date, name)
    
    def card_frame(self, date=slice(None, None, None), colors=slice(None, None, None), name=slice(None, None, None)):
        return self.ACTIVE_DATA.CARD_FRAME.card_frame(date, colors, name)

In [5]:
manager = DataManager()
manager.ACTIVE_SET = 'NEO'
manager.ACTIVE_FORMAT = 'PremierDraft'

In [6]:
manager.ACTIVE_DATA.get_summary_data()

Getting overall data for NEO PremierDraft
Updating data for 'CardRatings.json'. Fetching from 17Lands site...
Updating data for 'WCardRatings.json'. Fetching from 17Lands site...
Updating data for 'UCardRatings.json'. Fetching from 17Lands site...
Updating data for 'BCardRatings.json'. Fetching from 17Lands site...
Updating data for 'RCardRatings.json'. Fetching from 17Lands site...
Updating data for 'GCardRatings.json'. Fetching from 17Lands site...
Updating data for 'WUCardRatings.json'. Fetching from 17Lands site...
Updating data for 'WBCardRatings.json'. Fetching from 17Lands site...
Updating data for 'WRCardRatings.json'. Fetching from 17Lands site...
Updating data for 'WGCardRatings.json'. Fetching from 17Lands site...
Updating data for 'UBCardRatings.json'. Fetching from 17Lands site...
Updating data for 'URCardRatings.json'. Fetching from 17Lands site...
Updating data for 'UGCardRatings.json'. Fetching from 17Lands site...
Updating data for 'BRCardRatings.json'. Fetching from 1

(                      Colors  Splash   Wins   Games  Win %
 Name                                                      
 Mono-color               1.0   False    801    1370  58.47
 Mono-color + Splash      1.0    True   1494    2660  56.17
 Two-color                2.0   False  51707   91142  56.73
 Two-color + Splash       2.0    True  13527   24881  54.37
 Three-color              3.0   False   2750    5314  51.75
 Three-color + Splash     3.0    True    588    1090  53.94
 Four-color               4.0   False     38      68  55.88
 Four-color + Splash      4.0    True     64     108  59.26
 Five-color               5.0   False      1       4  25.00
 All Decks                NaN   False  70970  126637  56.04,
      Colors  Splash  Wins  Games  Win %
 Name                                   
 W         W   False    95    161  59.01
 U         U   False   136    270  50.37
 B         B   False    76    112  67.86
 R         R   False   263    458  57.42
 G         G   False   231    369

In [ ]:
data_store.get_card_frame_slice(colors='UR', name='Voldaren Epicure')

In [ ]:
data_store.get_summary_frame_slice(name='All Decks')

In [ ]:
data_store.get_archetype_frame_slice(name='WU')

In [ ]:
indiv_card_frame = data_store.get_card_frame_slice(colors='', name='Voldaren Epicure')
indiv_card_frame

In [ ]:
rolling_indiv_card_frame = indiv_card_frame.rolling(window=7, min_periods=1).mean()
rolling_indiv_card_frame

In [ ]:
gih_wr = indiv_card_frame.T.loc['GIH WR']
gih_wr

In [ ]:
rolling_gih_wr = rolling_indiv_card_frame.T.loc['GIH WR']
rolling_gih_wr

In [ ]:
indexes = [x for x in range(1, len(gih_wr)+1)]
wrs = [gih_wr[x] for x in range(0, len(gih_wr))]
rol_wrs = [rolling_gih_wr[x] for x in range(0, len(rolling_gih_wr))]       

In [ ]:
plt.plot(indexes, wrs, 'r.', indexes, rol_wrs, 'r')

In [ ]:
frame = today[2][''].copy()
avg_wr = today[0].loc['All Decks']['Win %']
wu_avg_wr = today[1][today[1]['Colors'] == 'WU'][today[1]['Splash'] == False].iloc[0]['Win %']
#frame['Δ WR'] = None
frame = frame.drop(['# Seen', '# Picked', '# OH', '# GD', '# GIH', '# GND'], axis=1)
frame['(x-μ) GP WR'] = frame['GP WR'] - avg_wr
frame['(x-μ) GIH WR'] = frame['GIH WR'] - avg_wr

frame.sort_values(by=['(x-μ) GP WR'])
frame[frame['(x-μ) GP WR'] < frame['(x-μ) GIH WR']].sort_values(by=['(x-μ) GIH WR'])
frame['Spikiness'] =  frame['GIH WR'] - frame['GP WR']
frame